In [1]:
from datetime import datetime, timedelta
import pytz
import json
import requests
import pandas as pd
import time 
import numpy as np
from pandas.io.json import json_normalize
# from keboola import docker

In [2]:
with open('data.txt', 'r') as myfile:
    API_token=myfile.read().replace('\n', '')

parameters = {  'authtoken': API_token,
                'flag': 'internal'}

In [10]:
def get_data_projects(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint][0])
        df_tmp['parent'] = to_append
        
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)

def get_data_tasklists(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    df_tmp = json_normalize(req.json()[endpoint])
    df_tmp['parent'] = to_append
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)
        
def get_data_tasks(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint])
        df_tmp['parent'] = to_append
    
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)

def get_data_timelogs(target_df, endpoint, to_append, target_id = '', url = ''):
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint]['tasklogs'])
        df_tmp['parent'] = to_append
    
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)
        


In [4]:
url = 'https://projectsapi.zoho.eu/restapi/portal/20061034739/projects/'
req = requests.get(url=url, params = parameters)
projects_list = req.json()['projects']
projects_list = [projects_list[i]['id'] for i in range(len(projects_list))]

projects_info = pd.DataFrame(np.zeros((0,0)))
for i in projects_list:
    projects_info = get_data_projects(target_df = projects_info, target_id = i,
                             url = url, endpoint = 'projects',
                             to_append ='Keboola')  

In [5]:
tasklists_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(projects_info['link.tasklist.url']):
    tasklists_info = get_data_tasklists(target_df = tasklists_info, url = i,
                              endpoint = 'tasklists', to_append = projects_info['id'].iloc[counter])

In [6]:
tasks_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(tasklists_info['link.task.url']):
    tasks_info = get_data_tasks(target_df = tasks_info, url = i,
                                endpoint = 'tasks',
                                to_append = tasklists_info['id'].iloc[counter])

In [11]:
timelogs_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(tasks_info['link.timesheet.url']):
    timelogs_info = get_data_timelogs(target_df = timelogs_info, url = i,
                                endpoint = 'timelogs',
                                to_append = tasks_info['id'].iloc[counter])